<h1 style="text-align:center;font-size:36px">NOTEBOOK DE BAC A SABLE</h1>

In [1]:
%load_ext autoreload
%autoreload 2

In [4]:
import sys
sys.path.append(r'C:\Users\martin.schoreisz\git\otv\otv\Transfert_Donnees')
sys.path.append(r'C:\Users\martin.schoreisz\git\Outils\Outils\src')
import pandas as pd
import geopandas as gp
import numpy as np
import altair as alt
from shapely.geometry import Point
from shapely.ops import transform
import pyproj
import re, os
from datetime import time, datetime
from random import randrange, choices, randint
from pathlib import Path
import Connexions.Connexion_Transfert as ct
import Import_trafics as it
import Donnees_horaires as dh
import Donnees_sources as ds
from Outils import Outils as O
from Params import Bdd_OTV as pb


pd.set_option('display.max_colwidth', 50)
pd.set_option('display.max_rows', 500)

# donnees Tipi

In [ ]:
donneesMinutes = pd.read_csv(r'C:\Users\martin.schoreisz\Documents\temp\OTV\DIRA\tipi_alienor_raw.v2.csv', delimiter=',')

In [ ]:
# liste des stations TIPI
listIdCompteurTipi=donneesMinutes.PME_ID.unique()
# liste des stations OTV
with ct.ConnexionBdd('local_otv_boulot') as c: 
    listObsSupl  =pd.read_sql("select distinct id_comptag, id_cpt, obs_supl from comptage.compteur where gestionnai='DIRA'", c.sqlAlchemyConn)
listStationOtvBrut = listObsSupl.obs_supl.apply(lambda x : x.split(';')[1].split('station : ')[1] if not pd.isnull(x) and 'EMC' not in x and ';' in x else x).unique()
listStationOtvBrut2 = [e.split(',') for e in listStationOtvBrut if e and re.search('^M.*\.._.*$', e)]
listStationOtv = [a.strip().replace('_','') for b in listStationOtvBrut2 for a in b]

In [ ]:
# station TIPI non présentes dans OTV : 
StationTipiHorsOTV = [s for s in listIdCompteurTipi if s not in listStationOtv]
StationOtvHorsTipi = [s for s in listStationOtv if s not in listIdCompteurTipi]
StationOtvTipi = [s for s in listIdCompteurTipi if s in listStationOtv]

# TEST OUVERTUR FICHIERE TAGMASTER .DAT

# donnees CD16 comptages temporaires

In [ ]:
cd16Temp=ds.FIM(r'C:\Users\martin.schoreisz\Documents\temp\OTV\CD16\en_cours\CD16_Temporaires\14-5.fim', gest='CD16')

In [ ]:
cd16Temp.geoloc, cd16Temp.geom_l93.wkt

In [ ]:
geom_l93

In [ ]:
# parcourir un dossier, vérifier si le le FIM est geoloc et récupérer la geom si c'est le cas
dossierCptTemp=r'C:\Users\martin.schoreisz\Documents\temp\OTV\CD16\en_cours\CD16_Temporaires'
dicoDf={'fichier': [], 'geom': [], 'tmja': [], 'pc_pl': []}
with os.scandir(dossierCptTemp) as it:
    for e in it:
        if e.is_file() and e.name.lower().endswith('.fim'):
            cpt=ds.FIM(e, verifQualite='Message')
            if cpt.geoloc:
                dicoDf['fichier'].append(e.name)
                dicoDf['geom'].append(cpt.geom_l93)
                dicoDf['tmja'].append(cpt.tmja)
                dicoDf['pc_pl'].append(cpt.tmja)

In [ ]:
df=pd.DataFrame(dicoDf)
gdf=gp.GeoDataFrame(df, geometry=df.geom, crs='epsg:2154')
gdf.drop('geom', axis=1).to_file(r'C:\Users\martin.schoreisz\Documents\temp\OTV\CD16\testGeomTemp.shp')

In [ ]:
cpt.geom_l93.wkt

In [ ]:
wgs84Proj = pyproj.CRS('EPSG:4326')
l93Proj = pyproj.CRS('EPSG:2154')
project = pyproj.Transformer.from_crs(wgs84Proj, l93Proj).transform
geoloc=re.search('(?P<lat>(\-|\+)[0-9]{1,3}\.([0-9]{4}\.){2})(?P<long>(\-|\+)[0-9]{1,3}\.([0-9]{4}\.){2})', '+45.6021.6666.-000.2134.6333.')
longitude = geoloc.group('long')
latitude = geoloc.group('lat')
x_wgs84=float(re.sub('(\+|-)(.*\.)(.*)(\.)(.*)(\.)', '\g<1>\g<2>\g<3>\g<5>', longitude))
y_wgs84=float(re.sub('(\+|-)(.*\.)(.*)(\.)(.*)(\.)', '\g<1>\g<2>\g<3>\g<5>', latitude))
geom_wgs84=Point(y_wgs84, x_wgs84 )
geom_l93 = transform(project, geom_wgs84)

In [ ]:
geom_l93.wkt

In [ ]:
cpt.dfHoraire2Sens.pivot(index=['date', 'type_veh', 'fichier'], columns='heure', values='nbVeh').reset_index().rename(
    columns={k: v for k, v in zip(['date', 'type_veh']+[e for e in range(24)], ['jour', 'indicateur']+dh.attributsHoraire)})

In [ ]:
cpt.dfHoraire2Sens

# test donnees 6 minutes vikings

In [9]:
dossier = r'C:\Users\martin.schoreisz\Documents\temp\DreamTeam\LaRochelle\run3'
listeResultats = []
listFichier = [os.path.join(dossier, '201_VR'+os.sep+'VR_Tot.VIK'),
               os.path.join(dossier, '164_BE'+os.sep+'BE_Tot.VIK'),
               os.path.join(dossier, '195_BS'+os.sep+'BS_tot.VIK'),
               os.path.join(dossier, '196_VL'+os.sep+'VL_tot.VIK')]

In [10]:
def corrTypeVeh(ser, typeVehExistant, serSeuilPL, serSeuil2R):
    O.checkParamValues(typeVehExistant, ['2R', 'VL', 'PL'])
    if (not serSeuilPL and not serSeuil2R) or (typeVehExistant == '2R' and not serSeuil2R): 
        return typeVehExistant
    elif serSeuilPL and int(ser.split('S=')[1]) >= serSeuilPL:
        return 'PL'
    elif serSeuil2R and int(ser.split('S=')[1]) < serSeuil2R:
        return '2R'
    elif serSeuilPL and not serSeuil2R and int(ser.split('S=')[1]) < serSeuilPL:
        if typeVehExistant in ('2R', 'VL'):
            return typeVehExistant
        else : 
            return 'VL'
    elif serSeuil2R and not serSeuilPL and int(ser.split('S=')[1]) >= serSeuil2R:
        if typeVehExistant in ('PL', 'VL'):
            return typeVehExistant
        else : 
            return 'VL'
    else:
        return 'VL'


for f in listFichier:
    # ouverture du fichier
    vik = ds.Viking(f)
    # mise en forme des données
    poste = Path(f).parts[-2][-2:]
    print(f, poste)
    try:
        voie = poste.split('_')[1]
    except IndexError:
        voie = None
    # correction du SER
    if poste == 'BE':
        serSeuilPL = 850
        serSeuil2R = 280
    elif poste == 'BS':
        serSeuilPL = None
        serSeuil2R = None
    elif poste == 'VL':
        serSeuilPL = 1600
        serSeuil2R = 392
    elif poste == 'VR':
        serSeuilPL = 2200
        serSeuil2R = 165
    else:
        serSeuilPL = None
        serSeuil2R = None
    test = vik.dfFichier.copy()
    test['type_veh'] = test.apply(lambda x: corrTypeVeh(x.ser, x.type_veh, serSeuilPL, serSeuil2R), axis=1)
    # creation des données
    dfGroupe = ds.GroupeCompletude(ds.NettoyageTemps(test), '6T', vitesse=True)#.iloc[:100]
    listeResultats.append(dfGroupe.pivot(index=['date_heure'], columns=['type_veh'], values=['nbVeh', 'v50', 'v10', 'v85']).reset_index().assign(
        voie=voie, capteur=poste))

0002. 017.0002.  90.   1.  22.  10.  25.  13.  00.0300.1500.   1. +46.1734.0498.-001.1794.1832.
C:\Users\martin.schoreisz\Documents\temp\DreamTeam\LaRochelle\run3\201_VR\VR_Tot.VIK VR
0003. 017.0003.  90.   1.  22.  10.  25.  14.  00.0200.1100.   2. N46.  10.  27. 672.W001.  10.  40. 476.
C:\Users\martin.schoreisz\Documents\temp\DreamTeam\LaRochelle\run3\164_BE\BE_Tot.VIK BE
0004. 017.0002.  90.   1.  22.  10.  25.  14.  30.0350.1100.   2. N46.  10.  28. 374.W001.  10.  40. 566.
C:\Users\martin.schoreisz\Documents\temp\DreamTeam\LaRochelle\run3\195_BS\BS_tot.VIK BS
0017. 017.0001.  02.   1.  22.  10.  25.  12.  00.0500.1700.   2. N46.  10.  24. 348.W001.  10.  46. 020.
C:\Users\martin.schoreisz\Documents\temp\DreamTeam\LaRochelle\run3\196_VL\VL_tot.VIK VL


In [11]:
# concatenation et export
concat = pd.concat(listeResultats)

In [12]:
concat.fillna(0).assign(voie=concat.voie).sort_values(['capteur', 'date_heure']).to_csv(
    r'C:\Users\martin.schoreisz\Box\Cerema\60-SO\66-DM\6610-MIM\Donnees\1-ROUTIER\1 - Nouvelle-Aquitaine\17 - Charente-Maritime\2022\La_Rochelle\3eme_phase_synthese6min_V2.csv')

In [13]:
concat.fillna(0).assign(voie=concat.voie).sort_values(['capteur', 'date_heure'])

date_heure nbVeh                 v50                     \
type_veh                        2R   PL   TV   VL   2R   PL     TV     VL   
0        2022-10-26 00:00:00   0.0  0.0  2.0  2.0  0.0  0.0   34.0   34.0   
1        2022-10-26 00:06:00   0.0  0.0  2.0  2.0  0.0  0.0   38.5   38.5   
2        2022-10-26 00:12:00   0.0  0.0  0.0  0.0  0.0  0.0    0.0    0.0   
3        2022-10-26 00:18:00   0.0  0.0  1.0  1.0  0.0  0.0   37.0   37.0   
4        2022-10-26 00:24:00   0.0  0.0  0.0  0.0  0.0  0.0    0.0    0.0   
...                      ...   ...  ...  ...  ...  ...  ...    ...    ...   
16554    2023-01-02 23:24:00   0.0  0.0  0.0  0.0  0.0  0.0    0.0    0.0   
16555    2023-01-02 23:30:00   0.0  0.0  0.0  0.0  0.0  0.0    0.0    0.0   
16556    2023-01-02 23:36:00   0.0  0.0  0.0  0.0  0.0  0.0    0.0    0.0   
16557    2023-01-02 23:42:00   0.0  0.0  2.0  2.0  0.0  0.0  100.5  100.5   
16558    2023-01-02 23:48:00   0.0  0.0  1.0  1.0  0.0  0.0  118.0  118.0   

          v10                     v85                       voie capteur  
type_veh   2R   PL     TV     VL   2R   PL      TV      VL                
0         0.0  0.0   34.0   34.0  0.0  0.0   34.00   34.00  None      BE  
1         0.0  0.0   38.1   38.1  0.0  0.0   38.85   38.85  None      BE  
2         0.0  0.0    0.0    0.0  0.0  0.0    0.00    0.00  None      BE  
3         0.0  0.0   37.0   37.0  0.0  0.0   37.00   37.00  None      BE  
4         0.0  0.0    0.0    0.0  0.0  0.0    0.00    0.00  None      BE  
...       ...  ...    ...    ...  ...  ...     ...     ...   ...     ...  
16554     0.0  0.0    0.0    0.0  0.0  0.0    0.00    0.00  None      VR  
16555     0.0  0.0    0.0    0.0  0.0  0.0    0.00    0.00  None      VR  
16556     0.0  0.0    0.0    0.0  0.0  0.0    0.00    0.00  None      VR  
16557     0.0  0.0   98.5   98.5  0.0  0.0  102.25  102.25  None      VR  
16558     0.0  0.0  118.0  118.0  0.0  0.0  118.00  118.00  None      VR  

[66238 rows x 19 columns]

# Donnees Vitesse Bordeaux ville zone 30

In [ ]:
dfGroupe.to_csv(r'C:\Users\martin.schoreisz\Documents\temp\DreamTeam\P5_.csv')

In [ ]:
dfValide = ds.NettoyageTemps(vik.dfFichier)
frequence='6T'

In [ ]:
dfGroupTypeHeure=dfValide.set_index('date_heure').groupby([pd.Grouper(freq=frequence),'type_veh','sens'])['nbVeh'].count().reset_index().sort_values('date_heure')
#completude des données
#ajout des données horaires à 0 si aucun type de vehicules mesures
date_range=pd.date_range(dfGroupTypeHeure.date_heure.min(),dfGroupTypeHeure.date_heure.max(),freq=frequence)
# df de comparaison
dfComp=pd.DataFrame({'type_veh':['2R','PL','VL']}).assign(key=1).merge(pd.DataFrame({'date_heure':date_range}).assign(key=1), on='key').merge(
pd.DataFrame({'sens':['sens1','sens2']}).assign(key=1)).sort_values(['date_heure', 'type_veh','sens'])[['date_heure','type_veh','sens']]
# df des données amnquantes
dfManq=dfComp.loc[dfComp.apply(lambda x : (x['date_heure'],x['type_veh'],x['sens']) not in zip(dfGroupTypeHeure.date_heure.tolist(),dfGroupTypeHeure.type_veh.tolist(),
                  dfGroupTypeHeure.sens.tolist()), axis=1)].copy()
dfHeureTypeSens = pd.concat([dfManq,dfGroupTypeHeure],axis=0).fillna(0).sort_values(['date_heure', 'type_veh','sens'])
dfHeureTypeSens['jour']=dfHeureTypeSens.date_heure.dt.dayofweek
dfHeureTypeSens['jourAnnee']=dfHeureTypeSens.date_heure.dt.dayofyear
dfHeureTypeSens['heure']=dfHeureTypeSens.date_heure.dt.hour
dfHeureTypeSens['date']=pd.to_datetime(dfHeureTypeSens.date_heure.dt.date)
dfHeureTypeSens=pd.concat([dfHeureTypeSens.groupby(['date_heure','sens']).agg({'nbVeh':'sum','heure':lambda x : x.unique()[0], 
                    'jour':lambda x : x.unique()[0],'jourAnnee':lambda x : x.unique()[0],'date':lambda x : x.unique()[0]}
                    ).assign(type_veh='TV').reset_index(),dfHeureTypeSens], axis=0, sort=False).sort_values(['date_heure', 'type_veh'])

In [ ]:
dfGroupVtsHeure=pd.concat([dfValide.set_index('date_heure').groupby([pd.Grouper(freq=frequence),'sens']).agg(
            **{'v10': pd.NamedAgg(column='vitesse',aggfunc=lambda x : np.percentile(x,10)),
               'v50': pd.NamedAgg(column='vitesse',aggfunc=lambda x : np.percentile(x,50)),
               'v85': pd.NamedAgg(column='vitesse',aggfunc=lambda x : np.percentile(x,85))}).reset_index().sort_values('date_heure').assign(type_veh='TV'),
           dfValide.set_index('date_heure').groupby([pd.Grouper(freq=frequence),'sens', 'type_veh']).agg(
            **{'v10': pd.NamedAgg(column='vitesse',aggfunc=lambda x : np.percentile(x,10)),
               'v50': pd.NamedAgg(column='vitesse',aggfunc=lambda x : np.percentile(x,50)),
               'v85': pd.NamedAgg(column='vitesse',aggfunc=lambda x : np.percentile(x,85))}).reset_index().sort_values('date_heure')])
dfHeureTypeSensV=dfHeureTypeSens.merge(dfGroupVtsHeure, on=['date_heure','sens', 'type_veh'],how='left').sort_values(['date_heure', 'type_veh', 'sens'])

In [ ]:
dossier = r'C:\Users\martin.schoreisz\Documents\temp\DreamTeam\BdxVille30\donnees_source\P8_cour_de_l_Yser\Brut'
bins=[0, 20, 30, 40, 50, 60, 70, 80, 90, 110, 9999]
cpt = ds.ComptageDonneesIndiv(dossier, vitesse=True)
cpt.dfSemaineMoyenne.groupby(['sens', 'type_veh']).agg(
            **{'mjo': pd.NamedAgg(column='nbVeh', aggfunc=lambda x : round(sum(x)/5))}).reset_index().to_csv(
    r'C:\Users\martin.schoreisz\Documents\temp\DreamTeam\BdxVille30\donnees_source\P8_cour_de_l_Yser\SortiePluginQgis\P8_MJO.csv')

In [ ]:
cpt.dicoNbJours

In [ ]:
test = cpt.df2SensBase.copy()
test['classe_vitesse'] = pd.cut(cpt.df2SensBase.vitesse, bins)#.value_counts()
test['jour'] = test.date_heure.dt.dayofyear
test['jourSemaine'] = test.date_heure.dt.dayofweek

In [ ]:
groupe = test.groupby(['sens', 'type_veh', 'classe_vitesse', 'jourSemaine', 'jour']).nbVeh.sum().reset_index().sort_values(
    ['jour', 'classe_vitesse', 'type_veh', 'sens', 'jour'])
groupe.loc[groupe.jour==319]

In [ ]:
test.loc[test.jour==319].jourSemaine.unique()

In [ ]:
# initialisation et calcul des donnees
dossier = r'C:\Users\martin.schoreisz\Documents\temp\DreamTeam\BdxVille30\donnees_source\P8_cour_de_l_Yser\Brut'
cpt = ds.ComptageDonneesIndiv(dossier, vitesse=True)

pd.concat([cpt.df2SensBase.loc[cpt.df2SensBase.date_heure.dt.dayofweek.isin(range(5))].groupby(['sens', 'type_veh']).agg(
            **{'v10': pd.NamedAgg(column='vitesse',aggfunc=lambda x : np.percentile(x,10)),
               'v50': pd.NamedAgg(column='vitesse',aggfunc=lambda x : np.percentile(x,50)),
               'v85': pd.NamedAgg(column='vitesse',aggfunc=lambda x : np.percentile(x,85)),
               'mjo': pd.NamedAgg(column='nbVeh', aggfunc=lambda x : round(sum(x)/5))}}).reset_index(),
cpt.df2SensBase.loc[cpt.df2SensBase.date_heure.dt.dayofweek.isin(range(5))].groupby(['type_veh']).agg(
            **{'v10': pd.NamedAgg(column='vitesse',aggfunc=lambda x : np.percentile(x,10)),
               'v50': pd.NamedAgg(column='vitesse',aggfunc=lambda x : np.percentile(x,50)),
               'v85': pd.NamedAgg(column='vitesse',aggfunc=lambda x : np.percentile(x,85)),
               'mjo': pd.NamedAgg(column='nbVeh', aggfunc=lambda x : round(sum(x)/5))}}).reset_index().assign(sens='tout sens'),
cpt.df2SensBase.loc[cpt.df2SensBase.date_heure.dt.dayofweek.isin(range(5))].groupby(['sens']).agg(
            **{'v10': pd.NamedAgg(column='vitesse',aggfunc=lambda x : np.percentile(x,10)),
               'v50': pd.NamedAgg(column='vitesse',aggfunc=lambda x : np.percentile(x,50)),
               'v85': pd.NamedAgg(column='vitesse',aggfunc=lambda x : np.percentile(x,85)),
               'mjo': pd.NamedAgg(column='nbVeh', aggfunc=lambda x : round(sum(x)/5))}).reset_index().assign(type_veh='TV'),
pd.DataFrame({'sens': ['tout sens'], 'type_veh': ['TV']}).assign(
    v50=cpt.df2SensBase.loc[cpt.df2SensBase.date_heure.dt.dayofweek.isin(range(5))].vitesse.quantile(),
    v10=cpt.df2SensBase.loc[cpt.df2SensBase.date_heure.dt.dayofweek.isin(range(5))].vitesse.quantile(0.1),
    v85=cpt.df2SensBase.loc[cpt.df2SensBase.date_heure.dt.dayofweek.isin(range(5))].vitesse.quantile(0.85),
    mjo=round(cpt.df2SensBase.loc[cpt.df2SensBase.date_heure.dt.dayofweek.isin(range(5))].nbVeh.sum()/5)).to_csv(
    r'C:\Users\martin.schoreisz\Documents\temp\DreamTeam\BdxVille30\donnees_source\P8_cour_de_l_Yser\SortiePluginQgis\P8_indic_agrege_MJO.csv')

In [ ]:
round(cpt.df2SensBase.loc[cpt.df2SensBase.date_heure.dt.dayofweek.isin(range(5))].nbVeh.sum()/5)

In [ ]:
pd.concat([dfValide.set_index('date_heure').groupby([pd.Grouper(freq=frequence),'sens']).agg(
            **{'v10': pd.NamedAgg(column='vitesse',aggfunc=lambda x : np.percentile(x,10)),
               'v50': pd.NamedAgg(column='vitesse',aggfunc=lambda x : np.percentile(x,50)),
               'v85': pd.NamedAgg(column='vitesse',aggfunc=lambda x : np.percentile(x,85))}).reset_index().sort_values('date_heure').assign(type_veh='TV'),
           dfValide.set_index('date_heure').groupby([pd.Grouper(freq=frequence),'sens', 'type_veh']).agg(
            **{'v10': pd.NamedAgg(column='vitesse',aggfunc=lambda x : np.percentile(x,10)),
               'v50': pd.NamedAgg(column='vitesse',aggfunc=lambda x : np.percentile(x,50)),
               'v85': pd.NamedAgg(column='vitesse',aggfunc=lambda x : np.percentile(x,85))}).reset_index().sort_values('date_heure')])

# affectation auto des points de comptage
> on part des points de comptages nouveau avec une géométrie (exemple dans le 47)<br>
***EN COURS<BR>CHAPITRE 6.2.2 ET 2CRITURE DE LA FONCTION creerComptageAssoc(). L'IDEE EST DE CREER LES DF DIRECTEMENT TELEVERSABLES DANS LA BDD.<br>ENSUITE IL FAUDRA VERIFIER QUE TOUTE LES ETAPES DU SCHEMA [geolocaliserPtComptage](C:\Users\martin.schoreisz\Box\Dossier_Perso\OTV\travail\methode\Schemas_OTV.mdj) SONT BIEN REALISEE. JE PENSE QUE LE PB A L'EXECUTION DE LA FONCTION VIENT DE LA FONCTIO MERGE QUI CREE DES _x ET _y SUR DES ATTRIBUTS QUE JE SOUHAITE REFERENCER ENSUITE***


In [ ]:
# recupérer les données
# params : nom de table  ou df des points geolocalises, nom de table de linauto, nom de Bdd
with ct.ConnexionBdd('local_otv_boulot') as c:
    pt47 = gp.read_postgis('select * from cd47_2020', c.sqlAlchemyConn)
    pt47Geom = gp.read_postgis('select * from cd47_2020 where geom is not null', c.sqlAlchemyConn)
    cptSansGeom = gp.read_postgis('select * from cd47_2020 where geom is null', c.sqlAlchemyConn)
    linauto47 = gp.read_postgis("""
                                 select DISTINCT ON (t.gid) c.id_comptag id_comptag_bdd,c.type_poste type_poste_bdd, t.gid, t.geom, st_distance(c.geom, t.geom)
                                  from (SELECT * FROM linauto.traf2020_bdt_na_ed20_simpli_l where list_dept like '%%47%%') t LEFT JOIN comptage.compteur c ON st_dwithin(c.geom, t.geom, 30)
                                  ORDER BY t.gid, CASE WHEN c.type_poste = 'permanent' THEN 1 
                                                       WHEN c.type_poste = 'tournant' THEN 2
                                                       WHEN c.type_poste = 'ponctuel' THEN 3 
                                                       ELSE 4 
                                                       end
                                 """, c.sqlAlchemyConn)

## Prealable
> ici on va preparer les donnes et fonctions generiques

### preparation des donnees

In [ ]:
# on en déduit le plus proche voisin entre les points et la linauto
ppV = O.plus_proche_voisin(pt47Geom, linauto47, 10, 'id_comptag', 'gid')
ppVSectHomo = ppV.loc[~ppV.gid.isna()].copy()
ppvHorsSectHomo = ppV.loc[ppV.gid.isna()].copy()  # PENSER A CREER LES COMPTEURS POUR CEUX LA !!!!!
# et on récupère l'id_comptag_existant et l'id de section homogene
ppVTot = pt47Geom.merge(ppVSectHomo, how='left', on='id_comptag').merge(linauto47, how='left', on='gid')

# trouver le nombre de comptage par section homogène (gid)
nbCptSectHomo = ppVTot.gid.value_counts()

# separer les differents cas a traiter
cptSimpleSectHomo = ppVTot.loc[ppVTot.gid.isin(nbCptSectHomo.loc[nbCptSectHomo == 1].index.tolist())].copy()
cptMultiSectHomo = ppVTot.loc[ppVTot.gid.isin(nbCptSectHomo.loc[nbCptSectHomo > 1].index.tolist())].copy()

### Fonctions génériques

In [ ]:
def creerCompteur(cptRef, attrGeom, dep, reseau, gestionnai, concession, techno=None, src_geo=None, obs_geo=None, obs_supl=None, id_cpt=None,
                  id_sect=None, fictif=False, en_service=True):
    """
    creation d'une df prete a etre integree dans la table des compteurs
    presque tout les chmaps 
    in : 
        cptRef : ou geodataframe avec un attribut de géométrie en SRID = 2154
        attrGeom : string : nom de l'attribut supportant la géométrie
        dep : departement sur 2 caractère ou pd.Series
        reseau : valeur depuis liste enumeree (cf bdd) ou pd.Series de ces valeurs
        gestionnai : valeur depuis liste enumeree (cf bdd) ou pd.Series de ces valeurs
        concession : boolean ou pd.Series de booleean
    """
    df = cptRef.copy()
    O.checkAttributsinDf(df, [attrGeom, 'id_comptag', 'type_poste', 'periode', 'pr', 'absc', 'route', 'src_cpt', 'convention', 'sens_cpt'])
    df = df[[attrGeom, 'id_comptag', 'type_poste', 'periode', 'pr', 'absc', 'route', 'src_cpt', 'convention', 'sens_cpt']].assign(
        dep=dep, reseau=reseau, gestionnai=gestionnai, concession=concession, techno=techno, src_geo=src_geo, obs_geo=obs_geo,
        obs_supl=obs_supl, id_cpt=id_cpt, id_sect=id_sect, fictif=fictif, en_service=en_service)
    df['x_l93'] = df[attrGeom].apply(lambda x: round(x.x,3))
    df['y_l93'] = df[attrGeom].apply(lambda x: round(x.y,3))
    df.drop(
        [c for c in df.columns if c not in ('dep', 'reseau', 'gestionnai', 'concession', attrGeom,
                                            'id_comptag', 'type_poste', 'periode', 'pr', 'absc', 'route', 'src_cpt', 'convention', 'sens_cpt', 'reseau', 'dep',
                                            'reseau', 'gestionnai', 'concession', 'techno', 'src_geo', 'obs_geo', 'obs_supl', 'id_cpt', 'id_sect', 'fictif',
                                            'en_service')], axis=1, inplace=True)
    gdfCpt = gp.GeoDataFrame(df, geometry=attrGeom, crs=2154)
    gdfCpt = O.gp_changer_nom_geom(gdfCpt, 'geom')
    return gdfCpt

In [ ]:
def rangBddComptageAssoc(id_comptag_ref_Bdd, bdd='local_otv_boulot'):
    """
    Récupérer dans la bdd, le rang max associé à un comptage de réference dans la table comptage_assoc.comptage
    """
    with ct.ConnexionBdd(bdd) as c:
        rqt = f"""SELECT COALESCE(max(cac.rang),0) rang 
                   FROM comptage_assoc.comptage cac JOIN comptage.comptage c ON c.id = cac.id_cptag_ref 
                   WHERE c.id_comptag='{id_comptag_ref_Bdd}'"""
        rang = pd.read_sql(rqt, c.sqlAlchemyConn).iloc[0]
    return rang

def creerComptageAssoc(df, id_comptag_ref_nom, annee, id_compteur_asso_nom, src=None, listIdCptExclu=None):
    """
    a partir d'une df, creer la df a injecter dans les tables compteur et comptage du schema comptage_assoc de la bdd
    in : 
        df : la df de base, doit contenir obligatoirement les attributs decrivant l'id_comptag_ref, le rang, le type_veh. si possible la periode, src, obs.
             si le comptage_assoc comprend une dimension géométrique, les attributs decrivant l'id_compteur_asso, le type_poste, la src_geo, la src_cpt, convention,
             et sens_cpt sont obligatoire. si possible ajouter route, pr, abs, techno, obs_geo, obs_supl
        id_comptag_ref_nom : string : nom de l'attribut supportant l'id_comptag du point de référence
        id_compteur_asso_nom : string : nom de l'attribut supportant l'id_comptag du point associé si composante geometrique (i.e l'id_comptag issu des donnees gestionnaire)
        annee : string annee sur 4 caractères
        listIdCptExclu : liste d'identifiant de comptage a ne pas conserver dans les resultats
    out : 
        dfIds : df contenant tous les points de la df en entree, avec les attributs issus de la Bdd en plus
        tableComptage : df au format de la table des comptage Associes. ne contient que les points 
    """
    def remplirObsSelonVacances(obsExistant, vacances):
        """
        si unn champs obs existe deja, on ajoute un phrase sur le fait que le point a été réalisé pendant les vacances, sinon on 
        ne met ue la menstion relatives aux vacances
        in : 
            obsexistant : strin ou valeur nulle
            vacances : boolean
        out :
            string
        """
        if not pd.isnull(obsExistant):
            if vacances:
                return f'{obsExistant} ; une partie des mesures sont réalisées pendant les vacances scolaires'
        else:
            if vacances:
                return 'une partie des mesures sont réalisées pendant les vacances scolaires'
        return None
    
    # mise en forme
    dfSource = df.copy()
    O.checkAttributsinDf(dfSource, ['type_veh', 'periode'])
    if listIdCptExclu:
        dfSource = dfSource.loc[~dfSource[id_compteur_asso_nom].isin(listIdCptExclu)].copy()
    dfSource.rename(columns={id_compteur_asso_nom: 'id_cpteur_asso'}, inplace=True)
    instanceCpt = it.Comptage('fake')
    corresIdComptagInterne = instanceCpt.recupererIdUniqComptage(dfSource[[id_comptag_ref_nom]].rename(columns={id_comptag_ref_nom: 'id_comptag'}), True)
    dfIds = dfSource.merge(corresIdComptagInterne, left_on=id_comptag_ref_nom, right_on='id_comptag', how='left').rename(columns={'id_comptag_uniq': 'id_comptag_ref'}).assign(annee=annee)
    dfIds['rang_bdd'] = dfIds[id_comptag_ref_nom].apply(lambda x: rangBddComptageAssoc(x))
    dfIds['rang_df'] = dfIds.groupby(id_comptag_ref_nom).cumcount()+1
    dfIds['rang'] = dfIds['rang_bdd']+dfIds['rang_df']

    tableComptage = dfIds[[c for c in pb.attrComptageAssoc if c in dfIds.columns]].copy()
    # ajouter ou modifier le champs observation pour les ponctuels ou tournant dont une partie de la periode est en vacances scolaire
    if 'obs' in tableComptage.columns:
        tableComptage['obs'] = tableComptage.apply(lambda x: remplirObsSelonVacances(x.obs, O.verifVacanceRange(x.periode)), axis=1)
    else:
        tableComptage['obs'] = tableComptage.periode.apply(lambda x: remplirObsSelonVacances(None, O.verifVacanceRange(x)))
    return dfIds, tableComptage


def creerCommpteurAssoc(df, nomAttrIdCpteurAsso, nomAttrGeom=None, nomAttrIdCpteurRef=None, listIdCptExclu=None):
    """
    a partir d'une df, creer la table des compteurdu schema comptage_assoc de la bdd. la table source doit contenir les attributs d'ientifiantde comptage,
    type_poste, src_geo, src_cpt, convention, sens_cpt. 
    pour plus de precision elle peut contenir geom, route, pr, abs, techno, obs_geo, obs_supl, id_cpt, id_sect, id_cpteur_ref. Cf Bdd pour plus de détail
    in : 
        df : dataframe des donnees sources
        nomAttrIdCpteurAsso : string : nom de l'attribut contenant les id_comptag mis en forme a partir du gestionnaire
        nomAttrIdCpteurRef : string : nom de l'attribut contenant les id_comptag de référence de la table compteur du schema comptapge de la bdd
        nomAttrGeom : string : nom de l'attribut qui supporte la géométrie
    out :
        dataframe au format bdd comptage_assoc.compteur
    """
    O.checkAttributsinDf(df, [nomAttrIdCpteurAsso, 'type_poste', 'src_geo', 'src_cpt', 'convention', 'sens_cpt'])
    dfSource = df.copy()
    if listIdCptExclu:
        dfSource = dfSource.loc[~dfSource[nomAttrIdCpteurAsso].isin(listIdCptExclu)].copy()
    dfSource.rename(columns={nomAttrIdCpteurAsso: 'id_cpteur_asso'}, inplace=True)
    if nomAttrIdCpteurRef:
        dfSource.rename(columns={nomAttrIdCpteurRef: 'id_cpteur_ref'}, inplace=True)
    if nomAttrGeom:
        dfSource = O.gp_changer_nom_geom(dfSource, 'geom')
    tableCompteur = dfSource[[c for c in dfSource.columns if c in pb.attrCompteurAssoc]]
    return tableCompteur

In [ ]:
def creerCorrespComptag(df, nomAttrIdComptagGest, nomAttrIdComptagGti, listIdCptExclu):
    """
    creer une df a integrer dans la bdd a partir d'une df comprenant : un attribut id_comptag de la bdd et un attribut id_comptag issu des données gestionnaire.
    si la df commprend des type poste gestionnaire et bdd on peut ajouter le nom de ces attributs pour faire un test 
    in :
        df : df de base doit contenir un attribut id_comptag de la bdd et un attribut id_comptag issu des données gestionnaire
        nomAttrIdComptagGest : string : nom de l'attribut décrivant l'id_comptag issu des données gestionnaire
        nomAttrIdComptagGti : string : nom de l'attribut décrivant l'id_comptag de la bdd
        listIdCptExclu : list ou tuple : list des id_comptage à ne pas prendre en compte
    """
    return df.loc[~df[nomAttrIdComptagGest].isin(listIdCptExclu)][[nomAttrIdComptagGest, nomAttrIdComptagGti]].copy().rename(columns={nomAttrIdComptagGest: 'id_gest', nomAttrIdComptagGti: 'id_gti'})

## CAS 1 : 1 seul comptage sur une section homogène
on va séparer les cas où un id_comptag est déjà présent ou non

In [ ]:
cptSimpleSectHomoNew = cptSimpleSectHomo.loc[cptSimpleSectHomo.id_comptag_bdd.isna()].copy()
cptSimpleSectHomoOld = cptSimpleSectHomo.loc[~cptSimpleSectHomo.id_comptag_bdd.isna()].copy()

## CAS 2 : plusieurs comptages sur une section homogène
Il va falloir déterminer quel est le comptage de référence (cf doc)

pour pouvoir déterminer le comptage de référence il faut connaitre : 
1. les types de postes (priorité : permanent > tournant > ponctuel)
1. les périodes de comptages (permanent non concerné)
1. les vacances scolaires
1. les niveaux de TMJA
1. le niveau de pc_pl
1. un choix arbitraire

### fonction de choix d'un comptage de reference

In [ ]:
# creation de la fonction de hiérarchisation
def hierarchisationCompteur(typePoste, periode, tmja, pc_pl):
    """
    obtention d"'une note globale de hierarhisation, a partir des 3 fonction incluse'
    """
    def hierarchisationTypePoste(typePoste):
        """
        attribuer une valeur de 3*10**21, 2*10**21 ou 10**21 selon le type de poste
        in : 
            typePoste : string parmi 'permanent', 'tournant', 'ponctuel'
        """
        O.checkParamValues(typePoste, ('permanent', 'tournant', 'ponctuel'))
        if typePoste == 'permanent':
            return 3*10**15
        elif typePoste == 'tournant':
            return 2*10**15
        else:
            return 10**15

    def hierarchisationVacance(periode):
        """
        attribuer une valeur de 2*10**18 ou 10**18 selon qu'une partie de la mesure a été faie pendant les vacances scolaire
        in :
            periode : string sou forme YYYY/MM/DD-YYYY/MM/DD éventuellement séparé par des ' ; '
        """
        testVacances = O.verifVacanceRange(periode)
        if not periode or testVacances:
            return 10**12
        else:
            return 2*10**12

    def hierarchisationTrafic(tmja, pc_pl):
        """
        fournir une valeur qui concatene le trafic et le pc_pl, en favorisant le trafic
        """
        if not tmja or tmja <= 0: 
            return ValueError('le tmja doit etre une superieur a 0  non nulle')
        elif not pc_pl:
            return tmja * 1000
        else:
            return (tmja * 1000) + pc_pl

    return hierarchisationTypePoste(typePoste) + hierarchisationVacance(periode) + hierarchisationTrafic(tmja, pc_pl)

In [ ]:
# calcul des compteur de reference et associes
cptMultiSectHomo['note_hierarchise'] = cptMultiSectHomo.apply(lambda x: hierarchisationCompteur(x.type_poste, x.periode, x.tmja, x.pc_pl), axis=1)
cptRefMultiSectHomo = cptMultiSectHomo.loc[cptMultiSectHomo.groupby('gid').note_hierarchise.transform('max') == cptMultiSectHomo.note_hierarchise].sort_values('gid').copy()
cptAssocMultiSectHomo = cptMultiSectHomo.loc[cptMultiSectHomo.groupby('gid').note_hierarchise.transform('max') != cptMultiSectHomo.note_hierarchise].sort_values('gid').copy()
cptAssocMultiSectHomo = cptAssocMultiSectHomo.merge(cptRefMultiSectHomo[['gid', 'id_comptag']], on='gid', suffixes=(None, '_ref'))
# verif que tous les gid ont un cpt ref
if not cptAssocMultiSectHomo.loc[~cptAssocMultiSectHomo.gid.isin(cptRefMultiSectHomo.gid.unique())].empty:
    raise ValueError('un des comptage associe n\'a pas de comptage de reference')
if not len(cptRefMultiSectHomo)+len(cptAssocMultiSectHomo) == len(cptMultiSectHomo):
    raise ValueError('un ou plusieurs comptage n\'ont pas ete affecte comme reference ou associe')

In [ ]:
# on sépare les comptages de références selon qu'ils sont sur un id_comptag de la bdd ou non
cptRefMultiSectHomoNew = cptRefMultiSectHomo.loc[cptRefMultiSectHomo.id_comptag_bdd.isna()].copy()
cptRefMultiSectHomoOld = cptRefMultiSectHomo.loc[~cptRefMultiSectHomo.id_comptag_bdd.isna()].copy()

## Comptage de référence sur section_homogene sans id_comptag
creation des compteurs mais validation finale apres analyse des donnes sur section homogene avec id_comptag

In [ ]:
# creation des compteurs
dfCompteurNewBruteAgrege = pd.concat([cptSimpleSectHomoNew, cptRefMultiSectHomoNew])

# A CE STADE ON NE CREE PAS ENCORE LES COMPTEURS ET COMPTAGES : ON VERIFIE D'ABORD QU'IL FAUT PAS EN AJOUTER CAR MAL CLASSES DANS D'AUTRE CATEGORIES

## Comptage de référence sur section_homogene avec id_comptag
selon la nature du comptage deja présent dans la base et selon la nature du comptage a injecter, on va soit : 
- creer un nouveau comptage associe avec le nouveau point
- creer une correspondance dans corresp_id_comptag
- modifier le type de poste et tracer la modif
- décider après analyse manuelle de creer un nouveau compteur

In [ ]:
def ventilerNouveauComptageRef(df, nomAttrtypePosteGest, nomAttrtypePosteBdd, nomAttrperiode):
    """
    depuis une df des comptage de référence situé sur des tronçons avec un id_comptag existantdans la base, 
    séparer en 3 groupe selon le type de poste dans la bdd et le type de poste du hestionnaire.
    in : 
        df : dataframe a classifier
        nomAttrtypePosteGest : string : nom de l'attribut supportant le type de poste fourni par le gest 
        nomAttrtypePosteBdd : string : nom de l'attribut supportant le type de poste dans la bdd
        nomAttrperiode : string : nom de l'attribut décrivant la période de mesure
    """
    dfCorrespIdComptag = df.loc[((df[nomAttrtypePosteGest] == df[nomAttrtypePosteBdd]) &
                                 (df[nomAttrtypePosteGest] != 'ponctuel')) |
                                ((df[nomAttrtypePosteGest] == df[nomAttrtypePosteBdd]) &
                                 (df[nomAttrtypePosteGest] == 'ponctuel') &
                                 (~df[nomAttrperiode].apply(lambda x: O.verifVacanceRange(x))))].copy()
    dfCreationComptageAssocie = df.loc[((df[nomAttrtypePosteGest] == 'ponctuel') &
                                        (df[nomAttrtypePosteBdd].isin(('permanent', 'tournant')))) |
                                       ((df[nomAttrtypePosteGest] == df[nomAttrtypePosteBdd]) &
                                        (df[nomAttrtypePosteGest] == 'ponctuel') &
                                        (df[nomAttrperiode].apply(lambda x: O.verifVacanceRange(x))))].copy()
    dfModifTypePoste = df.loc[((df[nomAttrtypePosteGest].isin(('permanent', 'tournant'))) & (df[nomAttrtypePosteBdd] == 'ponctuel')) |
                              ((df[nomAttrtypePosteGest] == 'permanent') & (df[nomAttrtypePosteBdd] == 'tournant'))].copy()
    return dfCorrespIdComptag, dfCreationComptageAssocie, dfModifTypePoste

In [ ]:
dfCompteurExistantAVentiler = pd.concat([cptSimpleSectHomoOld, cptRefMultiSectHomoOld])

In [ ]:
# pensez à vérifier d'éventuel pb de doublons du à la linauto en bordure de dept
dfCompteurExistantAVentiler.loc[dfCompteurExistantAVentiler.duplicated('index', keep = False)]

In [ ]:
# correction
dfCompteurExistantAVentiler.drop(34, inplace=True)

In [ ]:
dfCorrespIdComptag, dfCreationComptageAssocie, dfModifTypePoste = ventilerNouveauComptageRef(dfCompteurExistantAVentiler, 'type_poste', 'type_poste_bdd', 'periode')

In [ ]:
# analyse de la table de correspiIdComptag
# export pour vérif
gp.GeoDataFrame(dfCorrespIdComptag.drop('geom_y', axis=1), geometry='geom_x', crs=2154
                ).to_file(r'C:\Users\martin.schoreisz\Documents\temp\OTV\CD47\verifCorrespIdComptage.shp')

In [ ]:
# certains comptages doivent etre "forcé" en nouveau compteur. On fait une liste
listCompteurAForcer = []
listCompteurAForcer.append('47-D211-8+0')
listCompteurAForcer.append('47-D276-10+905')

In [ ]:
# analyse de la table de dfCreationComptageAssocie
# export pour vérif
gp.GeoDataFrame(dfCreationComptageAssocie.drop('geom_y', axis=1), geometry='geom_x', crs=2154
                ).to_file(r'C:\Users\martin.schoreisz\Documents\temp\OTV\CD47\verifComptagesAssoc.shp')

In [ ]:
# on ajoute les valeurs dans la liste des compteurs a forcer
listCompteurAForcer.append('47-D124-0+315')

In [ ]:
# analyse des modifs de type de poste : -> null
dfModifTypePoste

In [ ]:
# RESUME une ligne par donnee 
# correspIdComptag
# tableComptageAssoc
# tableCompteurAssoc

## creation des nouveaux compteurs

### recuperation des compteurs forcés depuis les comptages associés

In [ ]:
dfCompteurNewSupp = dfCompteurExistantAVentiler.loc[dfCompteurExistantAVentiler.id_comptag.isin(listCompteurAForcer)].copy()
dfCompteurNewBruteAgregeFinal = pd.concat([dfCompteurNewSupp, dfCompteurNewBruteAgrege])
attrGeomCompteurNew = 'geom_x'
dep = '47'
reseau = 'RD'
gestionnai = 'CD47'
concession = 'N'
dfCompteurNewBruteAgregeFinal['src_cpt'] = dfCompteurNewBruteAgregeFinal.type_poste.apply(lambda x: 'convention_gestionnaire' if x == 'permanent' else 'gestionnaire')
dfCompteurNewBruteAgregeFinal['convention'] = dfCompteurNewBruteAgregeFinal.type_poste.apply(lambda x: True if x == 'permanent' else False)
dfCompteurNewBruteAgregeFinal['sens_cpt'] = 'double sens'
dfCompteurNew = creerCompteur(dfCompteurNewBruteAgregeFinal, attrGeomCompteurNew, dep, reseau, gestionnai, concession, techno=None, 
                              src_geo=None, obs_geo=None, obs_supl=None, id_cpt=None, id_sect=None, fictif=False, en_service=True)

In [ ]:
# inserer dans la base

### creation des comptages

In [ ]:
# creation des comptages (il faut inserer les compteurs avant)
cptFictif = it.Comptage('faux')
cptFictif.creer_comptage(dfCompteurNewBruteAgrege.id_comptag.tolist(), 2020, 'donnees complete CD47', 'vl/pl', periode=dfCompteurNewBruteAgrege.periode.tolist())

### creation des indicateurs

In [ ]:
# creation des indicateurs (necessite la creation des comptages avant)
cptFictif.structureBddOld2NewForm(dfCompteurNewBruteAgrege.assign(annee='2020'), '2020', ['id_comptag','annee', 'fichier'], ['tmja', 'pc_pl'], 'agrege')

## creation des correspondances d'id_comptages

In [ ]:
correspIdComptag = creerCorrespComptag(dfCorrespIdComptag, 'id_comptag', 'id_comptag_bdd', listCompteurAForcer)

In [ ]:
# A INSERER EN BASE AVANT CREATION DES DONNEES 
correspIdComptag

## creation des comptages associés (avec tables compteurs et indics)

In [ ]:
# creer les comptages associés
dfIds, tableComptageAssoc = creerComptageAssoc(dfCreationComptageAssocie.assign(type_veh='vl/pl',
                                                           src_geo='pr+abs_gestionnaire',
                                                           convention=lambda x: dfCreationComptageAssocie.type_poste.apply(lambda x: True if x == 'permanent' else False),
                                                           sens_cpt='double sens',
                                                           src_cpt=lambda x: dfCreationComptageAssocie.type_poste.apply(lambda x: 'convention gestionnaire'
                                                                                                                        if x == 'permanent' else 'gestionnaire')),
                                                           'id_comptag_bdd', '2020', id_compteur_asso_nom='id_comptag', src=None, listIdCptExclu=listCompteurAForcer)

In [ ]:
# creer les compteurs relatifs aux comptages associes
tableCompteurAssoc = creerCommpteurAssoc(gp.GeoDataFrame(dfIds, geometry='geom_x', crs=2154).rename(
    columns={'absc': 'abs'}), 'id_cpteur_asso', 'geom_x', 'id_comptag_ref', listCompteurAForcer)

In [ ]:
# creer les indicateurs relatifs aux comptages agreges
# en fait il s'agit surtout de créer le lien entre l'id_comptag et l'id_cptag_uniq, sur le modele du schema comptage

In [ ]:
tableComptageAssoc, tableCompteurAssoc

# transfert des donnees raw TIPI dans la BDD
> on part du fichier transmis par l'equipe de Pascal GLASSON (C:\Users\martin.schoreisz\Box\Dossier_Perso\OTV\donnees_gestionnaires\DIRA\historique\2020\hors_convention\tipi_alienor_raw.v2.zip) 


In [ ]:
with ct.ConnexionBdd('local_otv_boulot') as c:
    ct.ogr2ogr_csv2pg(c.connstringOgr, r'C:\Users\martin.schoreisz\Documents\temp\OTV\DIRA\tipi_alienor_raw.v2.csv', table='dira_tipi_2020_raw')

# propositin de visualisation des niveaux d'émissions de polluants et GES (marche aussi pour le bruit)
> l'idée c'est de représenter la distance en absicsse et le niveau de pollution en ordonnée

In [3]:
# préparation des données
import altair as alt
import pandas as pd
from random import randint, choices
distance = [0] + sorted(choices(range(1, 3000),k=21))
nox = [randint(0, 10)]*5 + [randint(0, 10)]*3 + [randint(0, 10)]*4 + [randint(0, 10)]*2 + [randint(0, 10)]*6 + [randint(0, 10)]*2
particule = [randint(20, 100)]*5 + [randint(20, 100)]*3 + [randint(20, 100)]*4 + [randint(20, 100)]*2 + [randint(20, 100)]*6 + [randint(20, 100)]*2
bruit = [randint(60, 90)]*2 + [randint(60, 90)]*8 + [randint(60, 90)]*1 + [randint(60, 90)]*6 + [randint(60, 90)]*3 + [randint(60, 90)]*2
id_comptag = 'Géolocalisation Globale-dénomination de route-Géolocalisation fine'
dfNuisance = pd.DataFrame({'distance': distance, 'nox': nox, 'particule': particule, 'bruit': bruit})
dfNuisance = pd.melt(dfNuisance, id_vars='distance', var_name='type_nuisance')

# graph
listIndicateurs = dfNuisance.type_nuisance.unique()
input_dropdown = alt.binding_select(options=listIndicateurs, name='Nuisance')
selection = alt.selection_single(fields=['type_nuisance'], bind=input_dropdown)
alt.Chart(dfNuisance, title=['Nuisance des tronçons rattachés au compteur', id_comptag]).mark_line().encode(
    x='distance:O',
    y=alt.Y('value:Q', title='valeur (unité standard)'),
    color=alt.Color('Nuisance:N')
).add_selection(
    selection
).transform_filter(
    selection
)

alt.Chart(...)

In [5]:
dfNuisance.assign(id_comptag='Dept-Route-RepereGeo')

,distance,type_nuisance,value,id_comptag
0,0,nox,3,Dept-Route-RepereGeo
1,96,nox,3,Dept-Route-RepereGeo
2,127,nox,3,Dept-Route-RepereGeo
3,473,nox,3,Dept-Route-RepereGeo
4,483,nox,3,Dept-Route-RepereGeo
5,737,nox,1,Dept-Route-RepereGeo
6,1064,nox,1,Dept-Route-RepereGeo
7,1101,nox,1,Dept-Route-RepereGeo
8,1148,nox,9,Dept-Route-RepereGeo
9,1261,nox,9,Dept-Route-RepereGeo


# OTV 2020 : graph de résultats

In [68]:
fichier = r'C:\Users\martin.schoreisz\Documents\temp\OTV\livrable millesime 2020\2020_na_stat_pt_atmo\evol19_20_na_modif_titre.csv'
donneesSource = pd.read_csv(fichier, sep=';', usecols=[i for i in range(12)])
dfGestionnaire = donneesSource.loc[donneesSource.gestionnai.notna()].copy()
dfDepartement = donneesSource.loc[donneesSource.gestionnai.isna()].copy()
largeur = 500

In [66]:
titre = alt.TitleParams(["Évolution 2019 - 2020 selon le gestionnaire,", "l'importance et le département"],
                        fontSize=16)
(alt.Chart(dfGestionnaire,
           title=titre, width=largeur).mark_point(filled=True, size=120, stroke='black').encode(
     x=alt.X('gestionnai:N', title='Gestionnaire', axis=alt.Axis(labelAngle=45)),
     y=alt.Y('coeff_evol_19_20_moyen', axis=alt.Axis(format='%', title='Évolution moyenne 2019 - 2020')),
     color=alt.Color('importance:N', title='Importance (BdTopo)'),
     shape=alt.Shape('dept:O', title='Département'))
 .configure_axis(labelFontSize=12, titleFontSize=14)
 .configure_legend(labelFontSize=12, titleFontSize=14))

alt.Chart(...)

In [131]:
titre = alt.TitleParams(["Évolution 2019 - 2020 selon le département", "et l'importance"],
                        fontSize=16)
(alt.Chart(dfDepartement,
           title=titre, width=largeur).mark_point(filled=True, size=120, stroke='black').encode(
     x=alt.X('dept:N', title='Département', axis=alt.Axis(labelAngle=45)),
     y=alt.Y('coeff_evol_19_20_moyen', axis=alt.Axis(format='%', title='Évolution moyenne 2019 - 2020')),
     color=alt.Color('importance:N', title='Importance (BdTopo)'))
 .configure_axis(labelFontSize=12, titleFontSize=14)
 .configure_legend(labelFontSize=12, titleFontSize=14))

alt.Chart(...)

In [133]:
(alt.Chart(dfGestionnaire.assign(
    impact_corr_val_aberrantes=abs(dfGestionnaire.coeff_evol_19_20_moyen-dfGestionnaire.coeff_evol_19_20_moyen_hors_val_HS_SD)*100),
          title=alt.TitleParams(['impact de la suppression de valeur aberrantes', "sur l'évolution moyenne"],
                        fontSize=16),
          width=largeur)
 .mark_point(filled=True, size=120, stroke='black').encode(
     x=alt.X('gestionnai:N', title='Gestionnaire', axis=alt.Axis(labelAngle=45)),
     y=alt.Y('impact_corr_val_aberrantes', axis=alt.Axis(title=["Nombre de points de variation", "du pourcentage d'évolution"])),
     color=alt.Color('importance:N', title='Importance (BdTopo)'),
     shape=alt.Shape('dept:O', title='Département'))
 .configure_axis(labelFontSize=12, titleFontSize=14)
 .configure_legend(labelFontSize=12, titleFontSize=14))

alt.Chart(...)

# OTV : graph d'évolution annuelle

In [4]:
with ct.ConnexionBdd(pb.nomConnBddOtv) as c:
    df=pd.read_sql("select * from comptage.vue_evolutions_tmja where id_comptag = '33-A630-1+130'", c.sqlAlchemyConn)

In [5]:
alt.Chart(df).transform_fold(['evol_annuelle_globale', 'evol_anuelle'], ).mark_line().encode(
    x='annee',
    y='value:Q',
    color='key:N')

alt.Chart(...)

In [8]:
df

,id_unique_cpt,id_comptag,annee,tmja,tmja_min,tmja_max,tmja_recent,tmja_ancien,nb_annees_tot,nb_annees_prec,evol_anuelle,evol_annuelle_globale,evol_absolu_recente,variation_evolution_prec
0,58155,33-A630-1+130,2021,92087.0,79831.0,97895.0,79831.0,92540.0,11,1.0,15.35,-0.04,15.35,33.25
1,53641,33-A630-1+130,2020,79831.0,79831.0,97895.0,97239.0,92540.0,11,1.0,-17.90,-0.04,-17.90,19.06
2,36223,33-A630-1+130,2019,97239.0,79831.0,97895.0,96122.0,92540.0,11,1.0,1.16,-0.04,1.16,2.97
3,36222,33-A630-1+130,2018,96122.0,79831.0,97895.0,97895.0,92540.0,11,1.0,-1.81,-0.04,-1.81,3.37
4,36221,33-A630-1+130,2017,97895.0,79831.0,97895.0,96391.0,92540.0,11,1.0,1.56,-0.04,1.56,0.57
5,36220,33-A630-1+130,2016,96391.0,79831.0,97895.0,95450.0,92540.0,11,1.0,0.99,-0.04,0.99,1.09
6,36219,33-A630-1+130,2015,95450.0,79831.0,97895.0,93505.0,92540.0,11,1.0,2.08,-0.04,2.08,0.46
7,36218,33-A630-1+130,2014,93505.0,79831.0,97895.0,92015.0,92540.0,11,1.0,1.62,-0.04,1.62,3.45
8,36217,33-A630-1+130,2013,92015.0,79831.0,97895.0,93735.0,92540.0,11,1.0,-1.83,-0.04,-1.83,2.14
9,36216,33-A630-1+130,2012,93735.0,79831.0,97895.0,93445.0,92540.0,11,1.0,0.31,-0.04,0.31,0.67
